In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
len(train)

287113

In [2]:
train.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [3]:
training=train['article'][:2]
testing=train['highlights'][:2]

In [4]:
import tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer()
# random_sentence="12th Fail: Did you know IPS officer Manoj Kumar Sharma, IRS officer Shraddha Joshi had cameos in Vikrant Massey film?"
def tokenizenews(random_sentence):
    tokenizer=tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts([random_sentence])
    sequence = tokenizer.texts_to_sequences([random_sentence])
    sequence_length = 50  # Adjust as needed
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length)
    return padded_sequence
training=training.apply(tokenizenews)
testing=testing.apply(tokenizenews)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
from tensorflow.keras.models import Model
def encoder(array):
    vocab_size = 10000
    embedding_dim = 200
    encoder_hidden_states=[]
    sequence_length = 50
    input_sequence = tf.keras.Input(shape=(sequence_length, ))
    embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
    encoder_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1600 , return_sequences=True))(embedding_layer)
    encoded_representation = tf.keras.layers.Concatenate()([encoder_lstm[:, -1, :], encoder_lstm[:, 0, :]])
    model = tf.keras.Model(inputs=input_sequence, outputs=encoded_representation )
    for random_sentence in array:
        result = model.predict(random_sentence)
        print(result.shape)
        encoder_hidden_states.append(result)
    return encoder_hidden_states

In [6]:
def decoder(encoder_hidden_states,target_tokens,model1):
    loss = 0
    context_vector = None
    decoder_state = None
    sequence_length = 50
    hidden_size = 128
    embedding_dim = 128
    vocab_size = 10000
    context_vector=tf.zeros((128,50 ))
    decoder_state=tf.zeros((128, 50))
    learning_rate=0.001
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    v = tf.random.normal(shape=(hidden_size,50))
    Wh = tf.random.normal(shape=(hidden_size, 50))
    Ws = tf.random.normal(shape=(hidden_size, 50))
    battn = tf.random.normal(shape=(hidden_size,50))
    weights= tf.random.normal(shape=(100, 1))   
    bias=tf.random.uniform([]) 
    for t in range(len(target_tokens)):
        input_sequence = tf.keras.Input(shape=(sequence_length, ))
        embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
#         LSTM=tf.keras.layers.LSTM(128,return_sequences=True)(embedding_layer)
        model = tf.keras.Model(inputs=input_sequence, outputs=embedding_layer)
        output_embeddings = model.predict(target_tokens[t])
        attention_scores = []
        for i in range(len(encoder_hidden_states)):
            encoder_hidden_states[i] = tf.reshape(encoder_hidden_states[i], (128,50))
            e_t_i = v * tf.nn.tanh(Wh * encoder_hidden_states[i]+ Ws*decoder_state+  battn)
            attention_scores.append(e_t_i)
        attention_distributon=tf.nn.softmax(attention_scores)
        for i in range(len(encoder_hidden_states)):
            context_vector += attention_distributon[i] * encoder_hidden_states[i]
        output_embeddings=tf.reshape(output_embeddings, (128, 50, 1))
        output_embeddings=tf.squeeze(output_embeddings, axis=-1)
        concatenated_input = tf.expand_dims(tf.concat([context_vector,output_embeddings], axis=-1),axis=0)
#         output_embeddings1 = model1.predict(concatenated_input)
        output_embeddings1=tf.matmul(concatenated_input,weights)+bias
        vocab_distribution=tf.nn.softmax(output_embeddings1,axis=1)
        highest_prob_index=np.argmax(vocab_distribution)
        target_word_prob = vocab_distribution[0,highest_prob_index,0]
        timestep_loss = -tf.math.log(target_word_prob)
        decoder_state=(decoder_state+context_vector)/2
        decoder_state = tf.reshape(decoder_state, (128, 50))
        print(timestep_loss)
        loss += timestep_loss
        meanloss=loss/(t+1)
        #optimization process
        v+=learning_rate*meanloss* tf.nn.tanh(Wh * encoder_hidden_states[i]+ Ws*decoder_state+  battn)
        Ws+=learning_rate*meanloss*decoder_state
        Wh=Wh+learning_rate*meanloss*encoder_hidden_states[i]
        battn+=learning_rate*meanloss
        bias+=learning_rate*meanloss
        weights+=learning_rate*meanloss*weights
        decoder_state=(decoder_state+context_vector)/2
        decoder_state = tf.reshape(decoder_state, (128, 50))
    return [v,Ws,Wh,battn,weights,bias]
#         print(v,Ws,Wh,battn,decoder_state,model1.trainable_variables)
        

In [7]:
encoder_hidden=encoder(training)
# summarize="Germany, situated in central Europe, is renowned for its rich history, vibrant culture, and robust economy. With a population exceeding 80 million, it holds the highest population in the European Union. The country boasts picturesque landscapes like the Black Forest, Bavarian Alps, and Rhine Valley. Germany is a hub of technological advancements and innovation, housing top-tier universities and research institutions. It maintains well-developed infrastructure, efficient public transport, and a high standard of living. German cuisine, featuring hearty dishes like sausages, pretzels, and sauerkraut, is widely appreciated. Iconic landmarks such as the Brandenburg Gate, Neuschwanstein Castle, and Cologne Cathedral add to its allure. Offering a blend of historical sites and modern cities, Germany is a favored tourist destination and an attractive place for residence and employment."
target_tokens=[tokenizenews(train['highlights'][0])]
input_sequence1 = tf.keras.Input(shape=(128, 100))
layer2=tf.keras.layers.Dense(1)(input_sequence1)
model1 = tf.keras.Model(inputs=input_sequence1, outputs=layer2)
v,Ws,Wh,battn,weights,bias=decoder(encoder_hidden,testing,model1)

1/1 [==============================] - 1s 1s/step
(1, 6400)
1/1 [==============================] - 0s 21ms/step
(1, 6400)
1/1 [==============================] - 0s 35ms/step
tf.Tensor(4.3535457, shape=(), dtype=float32)
1/1 [==============================] - 0s 37ms/step
tf.Tensor(4.3976393, shape=(), dtype=float32)


In [8]:
def test(para,v,Ws,Wh,battn,weights,bias,length=50):
    tokenizer=tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts([para])
    sequence = tokenizer.texts_to_sequences([para])
    sequence_length = 50  # Adjust as needed
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=sequence_length)
    tokenspara=tokenizenews(para)
    print(tokenspara)
    encoder_hidden_states=encoder([tokenspara])
    summary=[]
    sequence_length = 50
    hidden_size = 128
    embedding_dim = 50
    vocab_size = 10000
    context_vector=tf.zeros((128,50 ))
    decoder_state=tf.zeros((128, 50))
    current_token = np.zeros((128,))
    for j in range(length):
        input_sequence = tf.keras.Input(shape=(sequence_length, ))
        embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_sequence)
#         LSTM=tf.keras.layers.LSTM(128,return_sequences=True)(embedding_layer)
        model = tf.keras.Model(inputs=input_sequence, outputs=embedding_layer)
        output_embeddings = model.predict(current_token)
        attention_scores = []
        for i in range(len(encoder_hidden_states)):
            encoder_hidden_states[i] = tf.reshape(encoder_hidden_states[i], (128,50))
            e_t_i = v * tf.nn.tanh(Wh * encoder_hidden_states[i]+ Ws*decoder_state+  battn)
            attention_scores.append(e_t_i)
        attention_distributon=tf.nn.softmax(attention_scores)
        for i in range(len(encoder_hidden_states)):
            context_vector += attention_distributon[i] * encoder_hidden_states[i]
#         print(output_embeddings.shape)
        output_embeddings=tf.reshape(output_embeddings, (128, 50, 1))
        output_embeddings=tf.squeeze(output_embeddings, axis=-1)
        concatenated_input = tf.expand_dims(tf.concat([context_vector,output_embeddings], axis=-1),axis=0)
#         output_embeddings1 = model1.predict(concatenated_input)
        output_embeddings1=tf.matmul(concatenated_input,weights)+bias
        vocab_distribution=tf.nn.softmax(output_embeddings1,axis=1)
        highest_prob_index=np.argmax(vocab_distribution)
        current_token[j]=highest_prob_index
#         print(highest_prob_index)
        decoder_state=(decoder_state+context_vector)/2
        decoder_state = tf.reshape(decoder_state, (128, 50))
        summary.append(current_token)
    tokentoword={tokenizer.word_index[i]:i for i in tokenizer.word_index.keys()}
#     print(tokentoword)
    return (" ".join([tokentoword[i]if i!=0 else " " for i in current_token]))

In [9]:
print(test(train['article'][100], v,Ws,Wh,battn,weights,bias,100))

[[ 62 209 210 211 212   2 213   3 214 215   1 216  41  35  15 217  36 218
   46 219 220   2  56  34 221   3  33 222  22   7   1 223  24   9   4 224
  225  20  14  10   2  34  12 226  17   9   4  13  18  11]]
1/1 [==============================] - 1s 686ms/step
(1, 6400)
4/4 [==============================] - 0s 2ms/step
extra   extra extra his extra extra time time time time extra time team time time during mascherano mascherano time before extra time sabella before time mascherano argentina mascherano extra before argentina's led holland out out argentina's up holland time out during right over up mates leader quiet argentina's leader video before before extra follow before during argentina video time with penalties match leader mascherano shoot out before before before mascherano match extra of out up with with paulo with before paulo mascherano leader paulo 90 time of video who mascherano of leader who who who quiet argentina match intently                                           